In [1]:
import gym
import torch, numpy as np
from torch import nn
import tianshou as ts
from agents import TwoAgentPolicy
from agents.lib_agents import SinePolicy
from utils import make_render_env, make_env, make_discrete_env, make_render_discrete_env

from tianshou.utils import TensorboardLogger, WandbLogger

In [2]:
# create environment

env = make_discrete_env()
train_envs = ts.env.DummyVectorEnv([make_discrete_env for _ in range(3)])
test_envs = ts.env.DummyVectorEnv([make_render_discrete_env for _ in range(5)])

/home/devanshu/.local/lib/python3.9/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [3]:
# setup network

class Net(nn.Module):
    def __init__(self, state_shape, action_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(state_shape), 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape)),
        )

    def forward(self, obs, state=None, info={}):
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float)
        batch = obs.shape[0]
        # print("obs", obs)
        # print("view", obs.view(batch, -1))
        logits = self.model(obs.view(batch, -1))
        return logits, state

In [4]:
state_shape = env.observation_space.shape or env.observation_space.n
puck_action_shape = env.action_space['puck'].shape or env.action_space['puck'].n
net1 = Net(state_shape, puck_action_shape)
bar_action_shape = env.action_space['bar'].shape or env.action_space['bar'].n
net2 = Net(state_shape, bar_action_shape)

In [5]:
optim1 = torch.optim.Adam(net1.parameters(), lr=1e-3)
optim2 = torch.optim.Adam(net2.parameters(), lr=1e-3)

In [6]:
# p1 = ts.policy.DQNPolicy(net1, optim1, discount_factor=0.9, estimation_step=3, target_update_freq=320)
# p1 = testPolicy()
p1 = SinePolicy()
p2 = ts.policy.DQNPolicy(net2, optim2, discount_factor=0.9, estimation_step=3, target_update_freq=320)
policy = TwoAgentPolicy(observation_space=env.observation_space, action_space=env.action_space, policies=(p1, p2))

In [7]:
# setup collector

train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(2000, 10), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)

In [8]:
# logging
logger = WandbLogger(
    save_interval=1,
    project="Penalty-Shot-Game",
    name='Devanshu Singla',
    entity='dsingla',
    run_id='2ea5f544f3091cb92327075643b98cf28aea2f3f'
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dsingla (use `wandb login --relogin` to force relogin)


In [9]:
# training

result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector,
    max_epoch=10, step_per_epoch=10000, step_per_collect=10,
    update_per_step=0.1, episode_per_test=100, batch_size=64)#, logger=logger)
print(f'Finished training! Use {result}')

Epoch #1:   0%|          | 0/10000 [00:00<?, ?it/s]/home/devanshu/.local/lib/python3.9/site-packages/tianshou/data/collector.py:183: UserWarning: n_step=10 is not a multiple of #env (3), which may cause extra transitions collected into the buffer.
  warnings.warn(
Epoch #1: 10008it [00:13, 764.24it/s, env_step=10008, len=0, loss=0.024, n/ep=0, n/st=12, rew=0.00]


Epoch #1: test_reward: 0.720000 ± 0.693974, best_reward: 0.720000 ± 0.693974 in #1


Epoch #2: 10008it [00:12, 791.72it/s, env_step=20016, len=0, loss=0.022, n/ep=0, n/st=12, rew=0.00]


Epoch #2: test_reward: 0.800000 ± 0.600000, best_reward: 0.800000 ± 0.600000 in #2


Epoch #3: 10008it [00:12, 800.05it/s, env_step=30024, len=0, loss=0.007, n/ep=0, n/st=12, rew=0.00]


...

Epoch #8: test_reward: -1.000000 ± 0.000000, best_reward: 1.000000 ± 0.000000 in #3


Epoch #9: 10008it [00:13, 766.08it/s, env_step=90072, len=0, loss=0.000, n/ep=0, n/st=12, rew=0.00]


Epoch #9: test_reward: -1.000000 ± 0.000000, best_reward: 1.000000 ± 0.000000 in #3


Epoch #10:  16%|#5        | 1560/10000 [00:01<00:10, 793.10it/s, env_step=91632, len=0, loss=0.000, n/ep=0, n/st=12, rew=0.00]


KeyboardInterrupt: 